In [1]:
# !pip install tensorflow-gpu keras-gpu

In [2]:
from math import log, floor
from sklearn.neighbors import KDTree

from pathlib import Path
from sklearn.utils import shuffle
from tqdm.notebook import tqdm as tqdm

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from itertools import cycle
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler

pd.set_option('max_columns', 50)
plt.style.use('bmh')
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

### Downcast function for efficient calculation

In [3]:
def downcast_dtypes(df):
  float_cols=[c for c in df if df[c].dtype == 'float64']
  int_cols = [c for c in df if df[c].dtype in ["int64","int32"]]
  df[float_cols]=df[float_cols].astype(np.float32)
  df[int_cols]=df[int_cols].astype(np.int16)
  return df

### Data load & preprocess

In [4]:
sales_train_validation = pd.read_csv('sales_train_validation.csv')
calendar = pd.read_csv('calendar.csv')
calendar = calendar.fillna(0)
sell_prices = pd.read_csv('sell_prices.csv')
sales_train_validation = pd.read_csv('sales_train_validation.csv')
sample_submission = pd.read_csv('sample_submission.csv')

sales_train_validation=downcast_dtypes(sales_train_validation)

### LSTM 모형 정의
* 3만개 품목에 대해서 출력이 3만개인 1개의 모형으로 fitting
* submission 데이터로 sMAPE 비교

Creating Dataset

In [1]:
sales_train_validation.head()

NameError: name 'sales_train_validation' is not defined

#### Data 생성을 위해 transpose 후 Id 관련 column 제거

In [6]:
sales_train_validation=sales_train_validation.T
sales_train_validations=sales_train_validation.fillna(0)
sales_train_validation=sales_train_validation[6:]

#### Data preprocess

In [7]:
sc = MinMaxScaler(feature_range=(0,1))
sales_train_validation=sc.fit_transform(sales_train_validation)

In [8]:
sales_train_validation.shape

(1913, 30490)

In [9]:
lookback = 20
x_train = []
y_train = []
for i in range(lookback, 1913):
    x_train.append(sales_train_validation[i-lookback:i])
    y_train.append(sales_train_validation[i][0:30490])

In [10]:
x_train=np.array(x_train, dtype='float16')
y_train=np.array(y_train, dtype='float16')

In [11]:
x_train.shape,y_train.shape

((1893, 20, 30490), (1893, 30490))

In [12]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14503870413973009985
xla_global_id: -1
]


In [13]:
model = keras.Sequential()
model.add(keras.layers.LSTM(units=64, input_shape=(np.array(x_train).shape[1], np.array(x_train).shape[2])))
model.add(keras.layers.Dense(30490))

model.compile(
    loss='mean_squared_error',
    optimizer = keras.optimizers.Adam(0.001)
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                7822080   
                                                                 
 dense (Dense)               (None, 30490)             1981850   
                                                                 
Total params: 9,803,930
Trainable params: 9,803,930
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(x_train, y_train, epochs=20, batch_size=10)

Epoch 1/20
190/190 [==============================] - 39s 189ms/step - loss: 0.0131
Epoch 2/20
190/190 [==============================] - 35s 186ms/step - loss: 0.0125
Epoch 3/20
190/190 [==============================] - 35s 185ms/step - loss: 0.0122
Epoch 4/20
190/190 [==============================] - 35s 184ms/step - loss: 0.0121
Epoch 5/20
190/190 [==============================] - 35s 182ms/step - loss: 0.0120
Epoch 6/20
190/190 [==============================] - 35s 182ms/step - loss: 0.0119
Epoch 7/20
190/190 [==============================] - 35s 182ms/step - loss: 0.0118
Epoch 8/20
190/190 [==============================] - 34s 181ms/step - loss: 0.0118
Epoch 9/20
190/190 [==============================] - 35s 182ms/step - loss: 0.0118
Epoch 10/20
190/190 [==============================] - 35s 184ms/step - loss: 0.0118
Epoch 11/20
190/190 [==============================] - 35s 183ms/step - loss: 0.0117
Epoch 12/20
190/190 [==============================] - 35s 183ms/step - lo

In [20]:
model.save('./model/lstm_model')

### Test Data
* 가장 마지막의 max_pred step을 예측
* 향후 submission의 첫 lookback period 이후 max_pred step을 예측

In [ ]:
lookback = 20
max_pred = 10
inputs = sales_train_validation[-max_pred - lookback: - max_pred]
actuals = sales_train_validation[- max_pred:]
inputs = sc.transform(inputs)
actuals = sc.transform(actuals)

In [ ]:
inputs_1 = np.expand_dims(inputs, 0)

In [ ]:
inputs_1.shape

(1, 20, 30490)

In [ ]:
actuals.shape

(10, 30490)

In [ ]:
predictions=model.predict(inputs_1)

In [ ]:
predictions.shape

(1, 30490)

#### Load model

In [1]:
from tensorflow import keras
model = keras.models.load_model('./model/lstm_model')

OSError: SavedModel file does not exist at: ./model/lstm_model/{saved_model.pbtxt|saved_model.pb}